## import modules & set variables

In [5]:
# for code implementation
import torch
import numpy as np
import torch.nn as nn
from google.colab import drive
from torch.optim import SGD, Adam
from torch.autograd import Variable
# for early stopping with IoU metric
from sklearn.metrics import jaccard_score
# for plotting graphs & data loader
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, CenterCrop, Normalize
from torchvision.transforms import ToTensor, ToPILImage
from torchvision.utils import make_grid
# others
from argparse import ArgumentParser
# variables - fixed for VOC2012 data set
NUM_CHANNELS = 3
NUM_CLASSES = 22

## mount drive, import custom modules
dataset.py, criterion.py, transform.py file from [provided link](https://github.com/bodokaiser/piwise)

In [ ]:
# mount your Google Drive
drive.flush_and_unmount()
drive.mount("/content/gdrive", force_remount=True)
# copy source files to this directory, to be able to import it directly
# if these files are copied and imported without any error,
# mounting is successful
!cp "gdrive/My Drive/Project/dataset.py" .
!cp "gdrive/My Drive/Project/criterion.py" .
!cp "gdrive/My Drive/Project/transform.py" .
!cp "gdrive/My Drive/Project/unet.py" .
from dataset import VOC12
from criterion import CrossEntropyLoss2d
from transform import Relabel, ToLabel, Colorize, colormap
import unet

## early stopping metrics
stop training early if we reach a certain point either with IoU or dice coefficient

In [7]:
def get_iou(labels, target):
    # receives 3D tensor (NxHxW) with values as the appropriate label
    np_labels = labels.cpu().numpy().reshape(-1)
    np_target = target.cpu().numpy().reshape(-1)
    # using 'weighted' may take a longer time than others
    return jaccard_score(np_target, np_labels, average='weighted')

def get_dice():
    raise NotImplementedError
    return 0

def early_stop(use_cuda, output, target, metric, threshold):
    '''
    output dim: 4x22x256x256
    target dim: 4x256x256
    given output and target, compute the metric (iou or dice)
    and return True if it exceeds the threshold value
    '''
    if metric == "iou":
        # output: reduce to 4x256x256 before passing to iou
        _, l = torch.max(output, dim=1)
        score = get_iou(labels=l, target=target)
        print("IoU metric score: {}".format(score))
        return (score > threshold)

    elif metric == "dice":
        score = get_dice()
        print("Dice metric score: {}".format(score))
        return (score > threshold)

    print("early_stop(): Incorrect metric... returning False by default")
    return False

## training
the basic structure of this function is also from [provided link](https://github.com/bodokaiser/piwise)

In [8]:
color_transform = Colorize()
image_transform = ToPILImage()
input_transform = Compose([
    CenterCrop(256), # 256x256 cropped image
    ToTensor(),
    Normalize([.485, .456, .406], [.229, .224, .225]), # mean, std for each channels
])

target_transform = Compose([
    CenterCrop(256), # 256x256 cropped image
    ToLabel(),
    Relabel(255, 21),
])
# target_transform = Compose([
#     CenterCrop(256),
#     ToTensor(),
#     Normalize((0),(1/256))
# ])
cmap = colormap(NUM_CLASSES)[:, np.newaxis, :]

def train(args, model):
    # set model to training mode
    model.train()
    # prepare criterion
    weight = torch.ones(22)
    weight[0] = 0
    # load data - even though Python notebook doesn't really support CLI arguments,
    # attributes in args will be assigned manually when calling main() function
    # previously, this was loading a big chunk.. we now divide into training set
    # and validation set
    train_loader = DataLoader(
                            VOC12(root=args.datadir, train=True, input_transform=input_transform,
                                  target_transform=target_transform),
                            num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)
    val_loader = DataLoader(
                            VOC12(root=args.datadir, train=False,input_transform=input_transform,
                                  target_transform=target_transform),
                            num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)
    # use Adam optimizer
    optimizer = Adam(model.parameters())
    # for loss calculation, we still use CrossEntropyLoss
    if args.cuda:
        criterion = CrossEntropyLoss2d(weight.cuda())
    else:
        criterion = CrossEntropyLoss2d(weight)
    # start training - epoch values start from 1 to make numbers look 'pretty'
    for epoch in range(1, args.num_epochs + 1):
        epoch_loss = []
        for step, (images, labels) in enumerate(train_loader):
            # check dim
            #print(images)
            #print(labels)
            if args.cuda:
                images = images.cuda()
                labels = labels.cuda()
            inputs = Variable(images)
            targets = Variable(labels)
            outputs = model(inputs)
            # refresh gradient before backprop
            optimizer.zero_grad()
            # print(targets[:, 0].shape)
            # print(outputs.shape)
            loss = criterion(outputs, targets[:, 0])
            loss.backward()
            optimizer.step()

            #epoch_loss.append(loss.data[0])
            epoch_loss.append(loss.item())
            if (args.steps_loss > 0) and (step % args.steps_loss == 0):
                # print loss
                average = sum(epoch_loss) / len(epoch_loss)
                print(f'loss: {average} (epoch {epoch}, step {step})')

                # check for early stop
                if early_stop(use_cuda=args.cuda, output=outputs, target=targets[:, 0], metric="iou", threshold=0.8):
                    # if we can stop early, save model and exit
                    print("Early stopping score exceeded threshold... saving model and ending training stage")
                    if args.attention:
                        filename = f'AttentionUNet-EarlyStop-{epoch:03}-{step:04}.pth'
                    else:
                        filename = f'UNet-EarlyStop-{epoch:03}-{step:04}.pth'
                    torch.save(model.state_dict(), filename)
                    return
            if (args.steps_save > 0) and (step % args.steps_save == 0):
                if args.attention:
                    filename = f'AttentionUNet-{epoch:03}-{step:04}.pth'
                else:
                    filename = f'UNet-{epoch:03}-{step:04}.pth'
                torch.save(model.state_dict(), filename)
                print(f'save: {filename} (epoch: {epoch}, step: {step})')

                # show images and how it's going so far
                # print('Image shape of trainloader : ', end='')
                # print(images.shape)
                # print('Label shape of train_loader : ', end='')
                # print(labels.shape)
                #print('Output shape of train_loader : ', end='')
                #print(outputs.shape)
                _, outputs = torch.max(outputs, dim=1)
                #print(outputs.shape)
                outputs = outputs.unsqueeze(1)
                #print(outputs.shape)
                # show images
                plt.subplot(211)
                plt.imshow(make_grid(images.cpu()).permute(1,2,0).numpy())
                plt.axis('off')
                plt.title('Images')
                # print labels
                plt.subplot(212)
                outputs = outputs.cpu().numpy()[:, :, :, :, np.newaxis]
                color_Label = np.dot(outputs == 0, cmap[0])
                for i in range(1, cmap.shape[0]):
                    color_Label += np.dot(outputs == i, cmap[i])
                color_Label = color_Label.swapaxes(1,4)
                plt.imshow((make_grid(torch.tensor(color_Label.squeeze())).permute(1,2,0).numpy()).astype('uint8'))
                plt.axis('off')
                plt.title('Label')

## evaluation

In [9]:
def evaluate(args, model):
    # set model to evaluation mode
    model.eval()

    image = input_transform(Image.open(args.image))
    label = model(Variable(image, volatile=True).unsqueeze(0))
    label = color_transform(label[0].data.max(0)[1])

    image_transform(label).save(args.label)

## main function


In [10]:
def main(args):
    # load correct model
    if args.attention == True:
        model = unet.AttentionUnet(in_channels=NUM_CHANNELS,out_channels=NUM_CLASSES)
    elif args.attention == False:
        # we don't use residual for non-denoising purposes
        model = unet.Unet(in_channels=NUM_CHANNELS,out_channels=NUM_CLASSES)
    # check if we use GPU
    if args.cuda:
        model = model.cuda()
    # check for mode - training or evaluation?
    if args.mode == 'eval':
        evaluate(args, model)
    elif args.mode == 'train':
        train(args, model)

## actual running
the basic structure of this snippet is also from [provided link](https://github.com/bodokaiser/piwise)

In [ ]:
if __name__ == '__main__':
    # comment out lines from original code that is not used here
    parser = ArgumentParser()
    parser.add_argument('--cuda', action='store_true')
    parser.add_argument('--attention', action='store_true')
    #parser.add_argument('--state')

    subparsers = parser.add_subparsers(dest='mode')
    subparsers.required = True

    parser_eval = subparsers.add_parser('eval')
    parser_eval.add_argument('image')
    parser_eval.add_argument('label')

    parser_train = subparsers.add_parser('train')
    #parser_train.add_argument('--port', type=int, default=80)
    parser_train.add_argument('--datadir', required=True)
    parser_train.add_argument('--num-epochs', type=int, default=32)
    parser_train.add_argument('--num-workers', type=int, default=4)
    parser_train.add_argument('--batch-size', type=int, default=1)
    parser_train.add_argument('--steps-loss', type=int, default=50)
    #parser_train.add_argument('--steps-plot', type=int, default=0)
    parser_train.add_argument('--steps-save', type=int, default=500)
    # since we can't pass CLI arugments in Python notebook,
    # assign variables manually
    # add two spaces between each chunk so that the data directory doesn't get split
    command_line = "--cuda  train  --datadir  gdrive/My Drive/Project/data  --num-epochs  320  --num-workers  4  --batch-size  4  --steps-save  500"
    args = parser.parse_args(command_line.split("  "))
    print(args)
    main(args)

In [ ]:
!ls sample_data

In [ ]:
!cp UNet-064-0500.pth "gdrive/My Drive/Project/models/UNetWithoutResPath-64epoch-1590.pth"